## **Import Libraries**

In [ ]:
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
import random

In [ ]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

random.seed(0)
weight_decay = 0.000005

In [ ]:
import abc
import itertools
import numpy as np
from keras.preprocessing.image import apply_affine_transform

## **Load and Preprocessing the data**

In [ ]:
def resize_and_crop_image(input_file, output_side_length, greyscale=False):
    img = cv2.imread(input_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB if not greyscale else cv2.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    new_height = output_side_length
    new_width = output_side_length
    if height > width:
        new_height = int(output_side_length * height / width)
    else:
        new_width = int(output_side_length * width / height)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    height_offset = (new_height - output_side_length) // 2
    width_offset = (new_width - output_side_length) // 2
    cropped_img = resized_img[height_offset:height_offset + output_side_length,
                              width_offset:width_offset + output_side_length]
    assert cropped_img.shape[:2] == (output_side_length, output_side_length)
    return cropped_img

In [ ]:
def normalize_minus1_1(data):
    return 2*(data/255.) - 1

In [ ]:
def get_channels_axis():
  import keras
  idf = keras.backend.image_data_format()
  if idf == 'channels_first':
      return 1
  assert idf == 'channels_last'
  return 3

In [ ]:
def load_cifar10():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(X_train))
    X_test = normalize_minus1_1(cast_to_floatx(X_test))
    return (X_train, y_train), (X_test, y_test)

## **Saving the results**

In [ ]:
def roc_pr_curve_data(scores, labels):
    scores = scores.flatten()
    labels = labels.flatten()

    scores_pos = scores[labels == 1]
    scores_neg = scores[labels != 1]

    truth = np.concatenate((np.zeros_like(scores_neg), np.ones_like(scores_pos)))
    preds = np.concatenate((scores_neg, scores_pos))
    fpr, tpr, roc_thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)
    print("roc_auc",roc_auc)

    # pr curve where "normal" is the positive class
    precision_norm, recall_norm, pr_thresholds_norm = precision_recall_curve(truth, preds)
    pr_auc_norm = auc(recall_norm, precision_norm)
    print("pr_auc_norm where normal is the positive class",pr_auc_norm)

    # pr curve where "anomaly" is the positive class
    precision_anom, recall_anom, pr_thresholds_anom = precision_recall_curve(truth, -preds, pos_label=0)
    pr_auc_anom = auc(recall_anom, precision_anom)
    print("pr_auc_norm where anomaly is the positive class",pr_auc_anom)

In [ ]:
def get_class_name_from_index(index, dataset_name):
    ind_to_name = {
        'cifar10': ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'),
    }

    return ind_to_name[dataset_name][index]

## **Transformations**

In [ ]:
class AffineTransformation(object):
    def __init__(self, flip, tx, ty, k_90_rotate):
        self.flip = flip
        self.tx = tx
        self.ty = ty
        self.k_90_rotate = k_90_rotate

    def __call__(self, x):
        res_x = x
        if self.flip:
            res_x = np.fliplr(res_x)
        if self.tx != 0 or self.ty != 0:
            res_x = apply_affine_transform(res_x, tx=self.tx, ty=self.ty,row_axis=0,
    col_axis=1, channel_axis=2, fill_mode='reflect')
        if self.k_90_rotate != 0:
            res_x = np.rot90(res_x, self.k_90_rotate)

        return res_x

In [ ]:
class AbstractTransformer(abc.ABC):
    def __init__(self):
        self._transformation_list = None
        self._create_transformation_list()

    @property
    def n_transforms(self):
        return len(self._transformation_list)

    @abc.abstractmethod
    def _create_transformation_list(self):
        return

    def transform_batch(self, x_batch, t_inds):
        assert len(x_batch) == len(t_inds)

        transformed_batch = x_batch.copy()
        for i, t_ind in enumerate(t_inds):
            transformed_batch[i] = self._transformation_list[t_ind](transformed_batch[i])
        return transformed_batch

In [ ]:
class Transformer(AbstractTransformer):
    def __init__(self, translation_x=8, translation_y=8):
        self.max_tx = translation_x
        self.max_ty = translation_y
        super().__init__()

    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, tx, ty, k_rotate in itertools.product((False, True),
                                                           (0, -self.max_tx, self.max_tx),
                                                           (0, -self.max_ty, self.max_ty),
                                                           range(4)):
            transformation = AffineTransformation(is_flip, tx, ty, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

In [ ]:
class SimpleTransformer(AbstractTransformer):
    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, k_rotate in itertools.product((False, True),
                                                    range(4)):
            transformation = AffineTransformation(is_flip, 0, 0, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

## **The Model: Wide Residual Network**

In [ ]:
def initial_conv(input):
    x = Convolution2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(input)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    return x


def expand_conv(init, base, k, strides=(1, 1)):
    x = Convolution2D(base * k, (3, 3), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = Convolution2D(base * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    skip = Convolution2D(base * k, (1, 1), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    m = Add()([x, skip])

    return m


def conv1_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv2_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv3_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    nb_conv = 4

    x = expand_conv(x, 16, k)
    nb_conv += 2

    for i in range(N - 1):
        x = conv1_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 32, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv2_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 64, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv3_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)

    x = Dense(nb_classes, kernel_regularizer=l2(weight_decay), activation='softmax')(x)

    model = Model(ip, x)

    if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model

## **Experiments**

In [ ]:
import os
import csv
from collections import defaultdict
from glob import glob
from datetime import datetime
from multiprocessing import Manager, freeze_support, Process
import numpy as np
import scipy.stats
from scipy.special import psi, polygamma
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
from tqdm.notebook import tqdm

In [ ]:
def _transformations_experiment(dataset_load_fn, dataset_name, single_class_ind, gpu_q):
    gpu_to_use = gpu_q.get()
    print(" Experiment for Class ",single_class_ind)
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_to_use

    #Load the data
    (x_train, y_train), (x_test, y_test) = dataset_load_fn()
    x_train = x_train[:TRAIN_SIZE]
    y_train =  y_train[:TRAIN_SIZE]
    x_test =  x_test[:VAL_SIZE]
    y_test =  y_test[:VAL_SIZE]

    #Transformations + Training the classifier
    transformer = Transformer(8, 8)
    n, k = (2, 8)
    mdl = create_wide_residual_network(x_train.shape[1:], transformer.n_transforms, n, k)
    mdl.compile('adam',
                'categorical_crossentropy',
                ['acc'])

    x_train_task = x_train[y_train.flatten() == single_class_ind]
    print("START the proposed transformations.................. ")
    transformations_inds = np.tile(np.arange(transformer.n_transforms), len(x_train_task))
    x_train_task_transformed = transformer.transform_batch(np.repeat(x_train_task, transformer.n_transforms, axis=0),
                                                           transformations_inds)
    print("END the proposed transformations......................... ")
    print("START the training of the classifier.................. ")

    mdl.fit(x=x_train_task_transformed, y=to_categorical(transformations_inds),
            batch_size=BATCH_SIZE, epochs= 5 #int(np.ceil(5/transformer.n_transforms)) #N_EPOCHS
            )
    print("END the training of the classifier.................. ")

    #Estimating the parameters
    def inv_psi(y, iters=5):
        # initial estimate
        cond = y >= -2.22
        x = cond * (np.exp(y) + 0.5) + (1 - cond) * -1 / (y - psi(1))

        for _ in range(iters):
            x = x - (psi(x) - y) / polygamma(1, x)
        return x     
    def quantity(y_j, p_i,a_i):
      return np.clip(np.sum((y_j/p_i)**(1/a_i)),10**(-5),10**3)
    def log_quantity(y_j, p_i,a_i):
      q = np.clip(y_j/p_i,10**(-8),10**3)
      return np.sum(np.log(q)*(q**(1/a_i)))
    def fixed_point_shifted_scaled_dir_mle(y,alpha_0,p_0,a_0,max_iter):
      N = y.shape[0]
      K = y.shape[1]
      y = np.asarray(np.clip(y,10**(-10),1))
      log_y = np.nan_to_num(np.log(np.clip(y,10**(-10),None)), copy=True, nan=1, posinf=10**10, neginf=-(10**10)).mean(axis=0)
      alpha_old = alpha_0
      p_old = p_0
      a_old = a_0
      alpha_new = np.zeros(K)
      p_new = np.zeros(K)
      for iter in range(max_iter):
        for i in range(K):
          #print('i=',i)
          #calculate vector alpha
          aph = psi(np.clip(np.sum(alpha_old),10**(-10),None)) + np.log(p_old[i]) + (1/a_old)*log_y[i] - (1/N)*np.sum([np.log(quantity(y[j],p_old,a_old)) for j in range(N)])
          aph = np.clip(aph,None,10)
          alpha_new[i] = inv_psi(aph, iters=5)
          #alpha_new[i] = np.nan_to_num(alpha_new[i], copy=True, nan=10**2, posinf=10, neginf=-(10**10))
          alpha_new[i] = np.clip(alpha_new[i],10**(-10),10**7)
          
          #calculate vector p
          p_new[i] = (alpha_old[i]/a_old)/(np.sum([(np.sum(alpha_old)*(y[j][i]**(1/a_old))*(1/a_old)*((1/p_old[i])**((1/a_old)+1)) ) / quantity(y[j],p_old,a_old) for j in range(N)]))
          p_new[i] = np.clip(p_new[i],10**(-10),None)
          #p_new[i] = np.nan_to_num(p_new[i], copy=True, nan=10**10, posinf=10, neginf=-(10**10))

          #calculate a
          a = np.sum([log_quantity(y[j],p_old,a_old)/quantity(y[j],p_old,a_old) for j in range(N)]) - N*np.sum(alpha_old*np.log(p_old)) - np.sum([np.sum(alpha_old*np.log(y)[j]) for j in range(N)])
          a_new = np.clip(a/(N*(K-1)),0.1,None)

        p_new = p_new/np.sum(p_new)
        if np.sqrt(np.sum((alpha_old - alpha_new) ** 2)) < 1e-2 and np.sqrt(np.sum((p_old - p_new) ** 2)) < 1e-2 and np.sqrt(np.sum((a_old - a_new) ** 2)) < 1e-2: 
          break
        #update the parameters
        alpha_old = alpha_new
        p_old = p_new
        a_old = a_new
      return alpha_new, p_new, a_new
  

    #Calculating the normality score
    def shifted_scaled_dirichlet_normality_score(alpha,p,a,y):
        N = y.shape[0]
        K = y.shape[1]
        y = np.asarray(np.clip(y,10**(-10),1))
        scores = np.asarray([np.sum(((alpha/a)-1)*np.log(y)[j]) - np.sum(alpha)*np.log(quantity(y[j],p,a)) for j in range(N)])
        scores = np.nan_to_num(scores, copy=True, nan=10**3, posinf=10**5, neginf=-(10**5))
        return scores 

    scores = np.zeros((len(x_test),))
    observed_data = x_train_task
    for t_ind in range(transformer.n_transforms):
        #print('transformation', t_ind)
        observed_dirichlet = mdl.predict(transformer.transform_batch(observed_data, [t_ind] * len(observed_data)),
                                         batch_size=BATCH_SIZE)
        
        alpha_0 = 0.2*np.ones(observed_dirichlet.shape[1])
        p_0 = 0.2*np.ones(observed_dirichlet.shape[1])
        a_0 = 1
        y = observed_dirichlet # Our matrix of observations 
        mle_alpha_t,mle_p_t, mle_a_t = fixed_point_shifted_scaled_dir_mle(observed_dirichlet,alpha_0,p_0,a_0,max_iter=10)
        mle_alpha_t= np.nan_to_num(mle_alpha_t, copy=True, nan=10, posinf=10**10, neginf=-10*10)
        mle_p_t= np.nan_to_num(mle_p_t, copy=True, nan=10, posinf=10**10, neginf=-10**10)
        mle_a_t= np.nan_to_num(mle_a_t, copy=True, nan=1, posinf=10**10, neginf=-10**10)
        x_test_p = mdl.predict(transformer.transform_batch(x_test, [t_ind] * len(x_test)),
                               batch_size=BATCH_SIZE)
        x_test_p = np.nan_to_num(x_test_p, copy=True, nan=0, posinf=None, neginf=None)
        scores += shifted_scaled_dirichlet_normality_score(mle_alpha_t,mle_p_t,mle_a_t,x_test_p)
  
    scores /= transformer.n_transforms
    labels = y_test.flatten() == single_class_ind
    roc_pr_curve_data(scores, labels)
    gpu_q.put(gpu_to_use)

In [ ]:
def run_experiments(load_dataset_fn, dataset_name, q, n_classes):
    print("START _transformations_experiment ")
    n_runs = 3
    # Transformations
    for i in range(n_runs):
        print('Run number:',i+1)
        processes = [Process(target=_transformations_experiment,
                             args=(load_dataset_fn, dataset_name, c, q)) for c in range(n_classes)]

        for p in processes:
            p.start()
        for p in processes:
            p.join()
    print("END _transformations_experiment ")

In [ ]:
RESULTS_DIR = ''
TRAIN_SIZE = 10000
VAL_SIZE = 1000
BATCH_SIZE = 128

In [ ]:
if __name__ == '__main__':
    freeze_support()
    N_GPUS = 1
    man = Manager()
    q = man.Queue(N_GPUS)
    for g in range(N_GPUS):
        q.put(str(g))

    experiments_list = [ 
        #(load_fashion_mnist, 'fashion-mnist', 10),
        (load_cifar10,'cifar10',10),
    ]

    for data_load_fn, dataset_name, n_classes in experiments_list:
        run_experiments(data_load_fn, dataset_name, q, n_classes)

START _transformations_experiment 
Run number: 1
 Experiment for Class  
0Wide Residual Network-16-8 created.
START the proposed transformations.................. 
END the proposed transformations......................... 
START the training of the classifier.................. 
Epoch 1/5
204/566 [=========>....................] - ETA: 31:29 - loss: 4.3243 - acc: 0.00 - ETA: 18:39 - loss: 4.3233 - acc: 0.01 - ETA: 18:32 - loss: 4.3219 - acc: 0.01 - ETA: 18:19 - loss: 4.3213 - acc: 0.01 - ETA: 18:44 - loss: 4.3212 - acc: 0.01 - ETA: 18:33 - loss: 4.3216 - acc: 0.01 - ETA: 18:25 - loss: 4.3219 - acc: 0.01 - ETA: 18:30 - loss: 4.3207 - acc: 0.02 - ETA: 18:22 - loss: 4.3195 - acc: 0.02 - ETA: 18:20 - loss: 4.3183 - acc: 0.02 - ETA: 18:16 - loss: 4.3181 - acc: 0.02 - ETA: 18:11 - loss: 4.3182 - acc: 0.02 - ETA: 18:07 - loss: 4.3177 - acc: 0.02 - ETA: 18:06 - loss: 4.3176 - acc: 0.02 - ETA: 18:05 - loss: 4.3173 - acc: 0.02 - ETA: 18:01 - loss: 4.3166 - acc: 0.02 - ETA: 17:57 - loss: 4.3162 - 